You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20361997308"> Bond Comparison BarPlot (12 min)</a>

In [1]:
import bql
import bqviz
import pandas as pd
import calendar

# instantiate bql service
bq = bql.Service()

In [2]:
# setup the data items
amt_out = bq.func.sum(bq.func.group(bq.data.amt_issued(), [bq.func.month(bq.data.issue_dt()), bq.func.year(bq.data.issue_dt())]))

# construct query universe
filter1 = bq.data.bics_level_2_industry_group_name() == 'Airlines'
filter2 = bq.data.issue_dt() > '2014-12-31'
univ = bq.univ.filter(bq.univ.debtuniv(types='ALL'), bq.func.and_(filter1, filter2))

# set additional params
params = {'mode':'cached','currency':'usd'}

# make make and execute bql request
bql_request = bql.Request(univ, {'amt_out':amt_out}, params)
bql_response = bq.execute(bql_request)
bql_data = bql_response[0].df()

# break out into current and prior df's
df_prior = bql_data.loc[bql_data['YEAR(ISSUE_DT())'] < 2020][['amt_out','MONTH(ISSUE_DT())']]
df_current = bql_data.loc[bql_data['YEAR(ISSUE_DT())'] == 2020][['amt_out','MONTH(ISSUE_DT())']]

# format df
df_current.reset_index(inplace=True)
df_current.set_index('MONTH(ISSUE_DT())',inplace=True)

# take average of monthly prior data
df_prior_monthly = df_prior.groupby('MONTH(ISSUE_DT())').mean()

# make new df containing both current and prior data
df_data = pd.concat([df_prior_monthly,df_current[['amt_out']]],sort=True,ignore_index=True, axis=1)
df_data['month'] = [calendar.month_abbr[x] for x in list(df_data.index)]

# reset index and rename cols
df_data.set_index('month', inplace=True)
df_data.rename(columns={0:'prior',1:"current"}, inplace=True)

# make and show viz
viz = bqviz.BarPlot(df_data).set_style()
viz.show()

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…